In [ ]:
import pandas as pd
import numpy as np
import config as cf
import matplotlib.pyplot as plt
import talib

In [ ]:
from datetime import datetime

### Loading data

In [ ]:
%store -r tickers

In [ ]:
tickers

In [ ]:
dfs = dict()
print("...loading...")
for ticker in tickers:
    symbol = ticker["symbol"]
    if str(symbol).endswith("USDT"):
        print(symbol, end=" ")
        filename = f"{cf.AGGREGATED_DATA_PATH}futures/{symbol}-4h.h5"
        df = pd.read_hdf(filename)
        dfs[symbol] = df

In [ ]:
len(dfs)

##### Pick AAAUSDT symbol which has history data length greater than 600 days

In [ ]:
len_list = []
coverage_list = []
last_list = []
TRADE_TIMEFRAME_IN_MS = 4 * 60 * 60 * 1000
for symbol in dfs.keys():
    len_list.append(len(dfs[symbol]))
    must_length = (dfs[symbol].index.max()-dfs[symbol].index.min())/TRADE_TIMEFRAME_IN_MS+1
    real_length = len(dfs[symbol])
    coverage_list.append(real_length/must_length)
    last_list.append(dfs[symbol].index.max())
stat_df = pd.DataFrame({"symbol": dfs.keys(), "length": len_list, "coverage":coverage_list, "last":last_list})

##### The Strategy is we arbitrage 5 pairs, with long top 5 that has the highest accumulated return in the last period, and short 5 pairs that has the lowest cummulated return in the last period.

##### Prepare data

In [ ]:
TRADE_TIMEFRAME_IN_MS = 4 * 60 * 60 * 1000

def preprocessing(trade_duration,lookback_period,holding_period,rsi_period,macd_fast,macd_slow,
                  macd_signal,macd_multiple,macd_short_multiple,stat_df,dfs):
    
    selected_symbols = stat_df[(stat_df["length"]>trade_duration+lookback_period+holding_period)&(stat_df["coverage"]==1)]["symbol"].values.tolist()
    for symbol in selected_symbols:
        df = dfs[symbol]
        df[f"{symbol}_log_return"] = np.log(df["Close"].pct_change()+1)
        df[f"{symbol}_rsi"] = talib.RSI(df["Close"], timeperiod=rsi_period)
        _,_,df[f"{symbol}_macd"] = talib.MACD(df["Close"],fastperiod=macd_fast,slowperiod=macd_slow,signalperiod=macd_signal) 
        df[f"{symbol}_future_return"] = df[f"{symbol}_log_return"].shift(-holding_period).rolling(holding_period).sum()
    
    sma_market_trend =  dfs["BTCUSDT"]["Close"].rolling(50).mean() - dfs["BTCUSDT"]["Close"].rolling(200).mean().copy()
    btc_price = dfs["BTCUSDT"]["Close"]
    
    _,_, bct_macd_trend = talib.MACD(dfs["BTCUSDT"]["Close"],fastperiod=macd_fast*macd_multiple,\
        slowperiod=macd_slow*macd_multiple,signalperiod=macd_signal*macd_multiple) 
    
    _,_, bct_macd_short_trend = talib.MACD(dfs["BTCUSDT"]["Close"],fastperiod=macd_fast*macd_short_multiple,\
        slowperiod=macd_slow*macd_short_multiple,signalperiod=macd_signal*macd_short_multiple) 

    sma_market_trend =  dfs["BTCUSDT"]["Close"].rolling(50).mean() - dfs["BTCUSDT"]["Close"].rolling(200).mean()
    btc_price = dfs["BTCUSDT"]["Close"]
    sma_market_trend.name = "SMA_market_trend"
    btc_price.name = "BTC_price"
    macd_market_trend = pd.Series(data = bct_macd_trend,index=dfs["BTCUSDT"].index,name="MACD_market_trend")
    macd_market_short_trend = pd.Series(data = bct_macd_short_trend,index=dfs["BTCUSDT"].index,name="MACD_market_short_trend")
    
    merged_df = pd.concat([dfs[symbol][[f"{symbol}_log_return",\
        f"{symbol}_future_return", f"{symbol}_rsi",f"{symbol}_macd"]] for symbol in selected_symbols] \
            +[sma_market_trend,macd_market_trend,macd_market_short_trend,btc_price],axis=1)
    
    merged_df.sort_index(ascending=True,inplace=True)
    # merged_df.fillna(method="ffill", inplace = True)
    # merged_df.dropna(inplace=True)

    return merged_df, selected_symbols

def calculate_discounted_sum(df,symbol,index,alpha,period):
    price_list = df.loc[index-period*TRADE_TIMEFRAME_IN_MS+1:index][f"{symbol}_log_return"].to_list()
    alpha_list = [alpha ** i for i in range(period-1,-1,-1)]
    return np.dot(price_list,alpha_list)

In [ ]:
def run_strategy(
        trade_duration:int,
        stat_df:pd.DataFrame,
        dfs:pd.DataFrame,
        lookback_period:int = 7 * 6,
        holding_period:int = 3 * 6,
        avoid_top:int = 3,
        n_pairs:int = 5,
        rsi_min:int = 25,
        rsi_max:int = 75,
        rsi_period:int = 14,
        macd_fast:int = 25,
        macd_slow:int = 12,
        macd_signal:int = 9,
        alpha:float = 0.97,
        verbose:int = 1,
        balance:int =100_000,
        trading_fee:float=0.0004,
        budget_ratio_by_market:float = 0.7,
        macd_multiple:int = 10,
        macd_short_multiple:int = 3
       ):
   
    print("...backtesting strategy...")
    
    merged_df, selected_symbols = preprocessing(
        dfs=dfs,
        stat_df=stat_df,
        trade_duration=trade_duration,
        lookback_period=lookback_period,
        holding_period=holding_period,
        rsi_period=rsi_period,
        macd_fast=macd_fast,
        macd_slow=macd_slow,
        macd_signal=macd_signal,
        macd_multiple=macd_multiple,
        macd_short_multiple=macd_short_multiple
        )
    
    trade_end = merged_df.index.max() - holding_period * TRADE_TIMEFRAME_IN_MS
    trade_start = max(trade_end - trade_duration * TRADE_TIMEFRAME_IN_MS,merged_df.index.min())
    
    #Check if any missing open-time index
    for i in range (trade_start,trade_end,TRADE_TIMEFRAME_IN_MS):
        if not i in merged_df.index:
            print(f"miss {i}")
    
    btc_init_price = merged_df.loc[trade_start,:]["BTC_price"]
    
    trade_df = pd.DataFrame({"open":[trade_start],"datetime":[""], "symbol":[""],"type":[""],"profit":[0],\
        "rsi":[0],"macd":[0],"cost":[0],"balance":[balance],"btc_price":[btc_init_price]})

    balance = balance
    profit_before_cost = 0
    
    for open_unix_time in range(trade_start,trade_end+1, holding_period * TRADE_TIMEFRAME_IN_MS):
        date_time = datetime.utcfromtimestamp(open_unix_time/1000).strftime('%Y-%m-%d %H:%M:%S')
        row = merged_df.loc[open_unix_time,:].copy()

        long_candidates = []
        short_candidates = []
        
        # Selecting long and short candidate   
        for symbol in selected_symbols:
            if ((row[f"{symbol}_rsi"] > 45) & (row[f"{symbol}_rsi"] < rsi_max) & (row[f"{symbol}_macd"] > 0)):
                long_candidates.append(symbol)
            
            if ((row[f"{symbol}_macd"] < 0) & (row[f"{symbol}_rsi"] < 55) & (row[f"{symbol}_rsi"] > rsi_min)):
                short_candidates.append(symbol)
        
        # Decide symbols to go long
        long_history_profits = dict()
        for symbol in long_candidates:
            profit = calculate_discounted_sum(merged_df,symbol,open_unix_time,alpha,lookback_period)
            long_history_profits[profit] = symbol
        sorted_long_symbol_list = sorted(long_history_profits,reverse=False)
        
        long_symbols = [long_history_profits[sorted_long_symbol_list[i]] \
            for i in range(min(len(long_candidates),n_pairs))]
        
        #Decide symbols to go short
        short_history_profits = dict()
        for symbol in short_candidates:
            profit = calculate_discounted_sum(merged_df,symbol,open_unix_time,alpha,lookback_period)
            short_history_profits[profit] = symbol
        sorted_short_symbol_list = sorted(short_history_profits,reverse=False)
        
        short_symbols = [short_history_profits[sorted_short_symbol_list[i]] \
            for i in range(min(len(short_candidates),n_pairs))]
        
        trade_symbols = []
        trade_types = []
        trade_costs = []
        trade_profits = []
        balances = []
        RSIs = []
        MACDs = []
        btc_prices = []

        trade_num = len(long_symbols)+len(short_symbols)
        invest_amount = balance/trade_num if trade_num > 0 else 0
        
        if (row["MACD_market_trend"] > 0) & (row["SMA_market_trend"] > 0) & (row["MACD_market_short_trend"] > 0) : 
            long_budget = budget_ratio_by_market * balance
            short_budget = balance - long_budget
        elif (row["MACD_market_trend"] < 0) & (row["SMA_market_trend"] < 0) & (row["MACD_market_short_trend"] < 0):
            long_budget = (1-budget_ratio_by_market) * balance
            short_budget = balance - long_budget
        else:
            long_budget = short_budget = 0.5 * balance
        
        long_invest_amount = long_budget/len(long_symbols) if len(long_symbols) > 0  else 0
        short_invest_amount = short_budget/len(short_symbols) if len(short_symbols) > 0 else 0
        
        for symbol in long_symbols:
            cumsum_log_return = merged_df.loc[open_unix_time,:][f"{symbol}_future_return"]
            profit_before_cost = long_invest_amount * (np.exp(cumsum_log_return)-1)
            cost = invest_amount * 2 * trading_fee
            profit = profit_before_cost - cost
            balance += profit #type: ignore
            trade_symbols.append(symbol)
            trade_types.append("long")
            trade_costs.append(cost)
            trade_profits.append(profit)
            RSIs.append(row[f"{symbol}_rsi"])
            MACDs.append(row[f"{symbol}_macd"])
            balances.append(balance)
            btc_prices.append(row["BTC_price"])
        
        for symbol in short_symbols:
            cumsum_log_return = merged_df.loc[open_unix_time,:][f"{symbol}_future_return"]
            profit_before_cost = short_invest_amount * -(np.exp(cumsum_log_return)-1)
            cost = invest_amount * 2 * trading_fee
            profit = profit_before_cost - cost
            balance += profit #type: ignore
            trade_symbols.append(symbol)
            trade_types.append("short")
            trade_profits.append(profit)
            trade_costs.append(cost)
            RSIs.append(row[f"{symbol}_rsi"])
            MACDs.append(row[f"{symbol}_macd"])
            balances.append(balance)
            btc_prices.append(row["BTC_price"])

        times = [open_unix_time for i in range(trade_num)]
        date_times = [date_time for i in range(trade_num)]

        row =pd.DataFrame({"open": times, "datetime": date_times, "symbol":trade_symbols,\
            "type":trade_types,"profit":trade_profits, "rsi": RSIs, "macd": MACDs, \
                "cost":trade_costs,"balance":balances,"btc_price":btc_prices})
        
        trade_df = pd.concat([trade_df,row],ignore_index=True)

        if verbose == 1:
            print(date_time)
            print(f"LONG {long_symbols}")
            print(f"SHORT {short_symbols}")
        elif verbose == 0:
            print(".", end = " ")
        else:
            raise ValueError("verbose: 0 or 1")
    
    trade_df["cumsum_profit"] = trade_df["profit"].cumsum()
    trade_df["cumsum_long_profit"] = trade_df[trade_df["type"] == "long"]["profit"].cumsum()
    trade_df["cumsum_short_profit"] = trade_df[trade_df["type"] == "short"]["profit"].cumsum()
    trade_df["cumsum_trading_cost"] = trade_df["cost"].cumsum()
    
    print("done")
    
    return trade_df

In [ ]:
def plot_results(trade_df):
    plt.figure(figsize=(16,9),dpi=720)
    plt.plot(trade_df["open"],trade_df["cumsum_profit"],c="b",linewidth=1,label="cumsum_profit")
    plt.plot(trade_df["open"],trade_df["cumsum_long_profit"],c="g",linewidth=1,label="cumsum_long_profit")
    plt.plot(trade_df["open"],trade_df["cumsum_short_profit"],c="r",linewidth=1,label="cumsum_short_profit")
    plt.plot(trade_df["open"],trade_df["cumsum_trading_cost"],c="y",linewidth=1,label="cumsum_trading_cost")
    
    highest = max(trade_df["cumsum_long_profit"].max(),trade_df["cumsum_long_profit"].max(),trade_df["cumsum_profit"].max())
    btc_highest = trade_df["btc_price"].max()
    
    plt.plot(trade_df["open"],trade_df["btc_price"]*highest/btc_highest,color="tab:gray",linewidth=1,label="BITCOIN")
    plt.plot(trade_df["open"],[0 for i in range(len(trade_df))],color="black",linewidth=1)
    plt.legend()
    plt.show()

#### Improved version
1. Allocation of long and short will be set based on market situation, do more long if bull and vise verse.
2. The holding period will be set dynamicallaly based on market situation. If more volatile, hold a bit shorter???

In [ ]:
trade_df = run_strategy(
    dfs = dfs,
    stat_df=stat_df,
    trade_duration = 6 * 700,
    lookback_period = 28,
    holding_period = 12,
    verbose = 0,
    alpha = 0.7,
    avoid_top = 0,
    n_pairs = 5,
    rsi_min = 30,
    rsi_max = 70,
    rsi_period = 2 * 14,
    macd_fast = 2 * 26,
    macd_slow = 2 * 12,
    macd_signal = 2 * 9,
    balance=5_000,
    trading_fee=0.0002,
    budget_ratio_by_market=0.75,
    macd_short_multiple=4,
    macd_multiple=10
)
plot_results(trade_df)